<a href="https://colab.research.google.com/github/wildmary/hello/blob/master/%D1%81%D0%BB%D0%BE%D0%B2%D0%B0%D1%80%D1%8C2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re # ну регулярки нам всегда нужны
import requests
import csv
from lxml import etree
import pandas as pd # пандас традиционно импортируют с алиасом pd, будем придерживаться
pd.__version__ # посмотрим версию -- вроде в колабе довольно свежая

'1.1.5'

In [33]:
print("one"+" "+"two")

one two


In [55]:
#part = pd.read_csv("part.csv", sep=";")
part = pd.read_csv("1part.csv", sep=";")
#partxlsx  = pd.read_excel("part.xlsx")
part.columns = ["Griechisch", "GLemma", "GMSA", "Slavisch", "SÜbersetzung", "SemantK", "Lateinisch", "LLemma", "LGram"]
part.columns = ["Griechisch", "GLemma", "GMSA", "Slavisch", "SÜbersetzung", "SemantK", "Lateinisch", "LLemma", "LGram"]

In [73]:
part
#part["SemantK"][12] = "тожде єже Βαθμίς"


,Griechisch,GLemma,GMSA,Slavisch,SÜbersetzung,SemantK,Lateinisch,LLemma,LGram
0,Hyper-/Metalemma,Lemma,МSА,Hyper-/Metalemma,Übersetzung,Semantischer Kommentar,Hyper-/Metalemma,Lemma,Gramm. Inf.
1,f. 148v (linke Spalte),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Β,Β Βῆτα,NaN,NaN,ꙏ,"ᲀ꙽торое єстᲆ писмо Ґреческое, єже Македонѧ|не ...",NaN,NaN,NaN
3,βᾶ,Βᾶ,NaN,NaN,NaN,Естᲆ гласᲆ себе ᲀзаимᲆ призивающиⷯ· таже гласе...,NaN,NaN,NaN
4,βάω,Βά,NaN,NaN,иди,NaN,ire,eas·,V 2. Ps. Sg. Konj. Präs. Akt.
5,NaN,NaN,NaN,NaN,стꙋпи,NaN,scandere,scande·,V Imp. Sg.
6,Βαάρας,Βαάρας,NaN,NaN,NaN,корень нѣкій,radix,radix quædam·,S Nom. Sg. F.
7,NaN,NaN,NaN,NaN,NaN,NaN,quidam,NaN,Pro Nom. Sg. F.
8,βαβάζειν,Βαβάζω,\μ ξω,NaN,кличꙋ,NaN,clamare,clamò·,V 1. Ps. Sg. Ind. Präs. Akt.
9,NaN,NaN,π χα/,NaN,ᲀᲆплю,NaN,vociferari,uociferor·,V 1. Ps. Sg. Ind. Präs. Med-pass.


In [ ]:
#tei = (open('tei.xml','w'))
#tei.write(etree.tostring(root, encoding="unicode"))
with open ('tei.xml', 'w', encoding = 'utf-8') as tei:  
  tei.write (etree.tostring(root, encoding='unicode'))

In [ ]:
table = []
for i, row in part[:12].iterrows():
  print("текущий", list(row))
  previous = list(row)
  table.append(previous)
  #print(table[0])
  if i == 0:
    continue
  else:
    table.remove(table[0])
  print("предыдущий", table[0][1])
 
  

[corr. -лѡⷩ]


In [134]:
root = etree.Element("TEI", xmlns="http://www.tei-c.org/ns/1.0") # создадим корневой тег
teiheader = etree.Element("teiHeader")
root.append(teiheader)
fileDesc = etree.SubElement(teiheader, "fileDesc")
titleStmt = etree.SubElement(fileDesc, "titleStmt")
title = etree.SubElement(titleStmt, "title")
title.text = 'Словарь Епифания Славинецкого'
text = etree.SubElement(root, "text")
body = etree.SubElement(text, "body")
for index, row in part[:33].iterrows():
  if index == 0:
    continue
  else:
   # print("следующий", part['SÜbersetzung'][index+1])
    if type(row["Griechisch"]) != float:
      if type(row["GLemma"]) == float and type(row["SÜbersetzung"]) == float and type(row["LLemma"]) == float:
        page_note()
      else:
      #print(row["Griechisch"])
        entry = etree.SubElement(body, "entry", xml_id=row["GLemma"], xml_lang="grc")
        form = etree.SubElement(entry, "form", type="hyperlemma")
        orth = etree.SubElement(form, "orth")
        orth.text = row["Griechisch"]
    #else: 
    if type(row["GLemma"]) != float:
      #print(row["GLemma"])
      if type(row["Griechisch"]) == float:
        entry = etree.SubElement(body, "entry", xml_id=row["GLemma"], xml_lang="grc")
      form = etree.SubElement(entry, "form", type="lemma")
      orth = etree.SubElement(form, "orth")
      orth.text = row["GLemma"]
      #print(type(row["GMSA"]))
      if type(row["GMSA"]) != float: #доделать!
        inflected = re.search("(.*?),\s(.*)", row["GMSA"])
        if inflected:
          inflected_gram()
      if type(row["SÜbersetzung"]) != float:
         sense_greek = etree.SubElement(entry, "sense", xml_id=row["GLemma"]+".1")
   #     print("текущий",row["GLemma"], row["SÜbersetzung"])
         slavic_latin_translation()
      several_slavic_equivalents_check()
    if type(row["SemantK"]) != float:
      semantischer_kommentar()
      



        



In [135]:
print(etree.tostring(root, pretty_print=True, encoding='unicode'))

<TEI xmlns="http://www.tei-c.org/ns/1.0">
  <teiHeader>
    <fileDesc>
      <titleStmt>
        <title>Словарь Епифания Славинецкого</title>
      </titleStmt>
    </fileDesc>
  </teiHeader>
  <text>
    <body>
      <cb pageside="148v" side="(linke">f. 148v (linke Spalte)</cb>
      <entry xml_id="Β Βῆτα" xml_lang="grc">
        <form type="hyperlemma">
          <orth>Β</orth>
        </form>
        <form type="lemma">
          <orth>Β Βῆτα</orth>
        </form>
        <sense xml_id="Β Βῆτα.1">
          <cit type="translationEquivalent" xml_lang="chu">
            <form xml_id="ꙏ">
              <orth>ꙏ</orth>
            </form>
          </cit>
        </sense>
        <sense xml_id="Β Βῆτα.1">
          <note>ᲀ꙽торое єстᲆ писмо Ґреческое, єже Македонѧ|не убо мѣстѡ φ· Делфѧне же мѣстѡ π полагаютᲆ· | Естᲆ же числителное знамѧ два знаменꙋющее·</note>
        </sense>
      </entry>
      <entry xml_id="Βᾶ" xml_lang="grc">
        <form type="hyperlemma">
          <orth>βᾶ</ort

In [ ]:
text = etree.SubElement(root, "text")


In [121]:
 def slavic_latin_translation():
   cit = etree.SubElement(sense_greek, "cit", type="translationEquivalent", xml_lang = "chu")
   form = etree.SubElement(cit, "form",  xml_id=row["SÜbersetzung"].strip(","))
   orth = etree.SubElement(form, "orth")
   orth.text = row["SÜbersetzung"].strip(",")
   if type(row["Lateinisch"]) != float and row["Lateinisch"] != "-":
        sense = etree.SubElement(cit, "sense", xml_id=row["SÜbersetzung"].strip(",")+".1")
        cit = etree.SubElement(sense, "cit", type="translationEquivalent", xml_lang = "lat")
        form = etree.SubElement(cit, "form", type="hyperlemma", xml_id=row["Lateinisch"])
        orth = etree.SubElement(form, "orth")
        if type(part['SÜbersetzung'][index+1]) == float \
        and type(part['Lateinisch'][index+1]) != float and part['Lateinisch'][index+1] != "-":
          orth.text = row["Lateinisch"]+" "+part['Lateinisch'][index+1]
        else:
          orth.text = row["Lateinisch"]
   if type(row["LLemma"]) != float and row["LLemma"] != "-":
    form = etree.SubElement(cit, "form", type="lemma", xml_id=row["LLemma"])
    orth = etree.SubElement(form, "orth")
    orth.text = row["LLemma"]

   
    

In [119]:
def inflected_gram():
  form = etree.SubElement(entry, "form", type="inflected")
  gramGrp = etree.SubElement(form, "gramGrp")
  gram = etree.SubElement(gramGrp, "gram", case="genitiv")
  orth = etree.SubElement(form, "orth", extent="part")
  orth.text = inflected.group(1)
  if inflected.group(2) == "ἡ":
     gramGrp = etree.SubElement(entry, "gramGrp")
     gram = etree.SubElement(gramGrp, "gram", gen="fem")
     gram.text = "ἡ"
  elif inflected.group(2) == "τὸ":
     gramGrp = etree.SubElement(entry, "gramGrp")
     gram = etree.SubElement(gramGrp, "gram", gen="neut")
     gram.text = "τὸ"
  else:
     gramGrp = etree.SubElement(entry, "gramGrp")
     gram = etree.SubElement(gramGrp, "gram", gen="masc")
     gram.text = "ὁ"

In [118]:
def several_slavic_equivalents_check():
  global index
  while type(part["GLemma"][index+1]) == float:
    if type(part['SÜbersetzung'][index+1]) != float:
      cit = etree.SubElement(sense_greek, "cit", type="translationEquivalent", xml_lang = "chu")
      form = etree.SubElement(cit, "form",  xml_id=part['SÜbersetzung'][index+1].strip(","))
      orth = etree.SubElement(form, "orth")
      orth.text = part['SÜbersetzung'][index+1].strip(",")
      if type(part['Lateinisch'][index+1]) != float and part['Lateinisch'][index+1] != "-":
        sense = etree.SubElement(cit, "sense", xml_id=part['SÜbersetzung'][index+1]+".1")
        cit = etree.SubElement(sense, "cit", type="translationEquivalent", xml_lang = "lat")
        form = etree.SubElement(cit, "form", type="hyperlemma", xml_id=part['Lateinisch'][index+1])
        orth = etree.SubElement(form, "orth")
        if type(part['SÜbersetzung'][index+2]) == float \
        and type(part['Lateinisch'][index+2]) != float and part['Lateinisch'][index+2] != "-": 
          orth.text = part['Lateinisch'][index+1]+" "+part['Lateinisch'][index+2]
          if type(part['SÜbersetzung'][index+3]) == float \
          and type(part['Lateinisch'][index+3]) != float and part['Lateinisch'][index+3] != "-":
            orth.text = part['Lateinisch'][index+1]+" "+part['Lateinisch'][index+2]+" "+part['Lateinisch'][index+3]
        else:
          orth.text = part['Lateinisch'][index+1]
      if type(part['LLemma'][index+1]) != float and part['LLemma'][index+1] != "-":
        if type(part['Lateinisch'][index+1]) == float or part['Lateinisch'][index+1] == "-":
          sense = etree.SubElement(cit, "sense", xml_id=part['SÜbersetzung'][index+1].strip(",")+".1")
          cit = etree.SubElement(sense, "cit", type="translationEquivalent", xml_lang = "lat")
        form = etree.SubElement(cit, "form", type="lemma", xml_id=part['LLemma'][index+1])
        orth = etree.SubElement(form, "orth")
        if type(part['SÜbersetzung'][index+2]) == float \
        and type(part['LLemma'][index+2]) != float and part['LLemma'][index+2] != "-": 
          orth.text = part['LLemma'][index+1]+" "+part['LLemma'][index+2]
          if type(part['SÜbersetzung'][index+3]) == float \
          and type(part['LLemma'][index+3]) != float and part['LLemma'][index+3] != "-":
            orth.text = part['LLemma'][index+1]+" "+part['LLemma'][index+2]+" "+part['LLemma'][index+3]
        else:
          orth.text = part['LLemma'][index+1]
    index+= 1

In [117]:
def page_note():
  column = re.search("^(?P<f>f.)\s(?P<page_num>\d\d\d.)[,\s|\s(](?P<side>.*)\s(?P<Spalte>Spalte)", row["Griechisch"])
  page = re.search("Kustode.*(?P<pg>\d\d\d.)", row["Griechisch"])
  if column:
    cb = etree.SubElement(body, "cb", pageside=column.group("page_num"), side=column.group("side"))
    cb.text = row["Griechisch"]
  if page:
    pb = etree.SubElement(body, "pb", n=page.group("pg"))
    pb.text = row["Griechisch"]
  if not column and not page:
    marg_note = etree.SubElement(body, "note")
    marg_note.text = row["Griechisch"]

In [136]:
def semantischer_kommentar():
  global index
  note_in_entry = re.search("(\[.*\])", row["SemantK"])
  cross_reference_mention = re.search("(тожде єже|тожде|таже)\s(.*)", row["SemantK"])
  sense_note = etree.SubElement(entry, "sense", xml_id=row["GLemma"]+".1")
  note = etree.SubElement(sense_note, "note")
  if row["SemantK"] == "тожде":
    xr = etree.SubElement(note, "xr", type="synonymy")
    while type(part["GLemma"][index-1]) == float:
      index-= 1
    ref = etree.SubElement(xr, "ref", type="entry", target="#"+part['GLemma'][index-1])
    ref.text = row["SemantK"]
  if cross_reference_mention:
    xr = etree.SubElement(note, "xr", type="synonymy")
    lbl = etree.SubElement(xr, "lbl")
    lbl.text = cross_reference_mention.group(1)
    ref = etree.SubElement(xr, "ref", type="entry", target="#"+cross_reference_mention.group(2))
    ref.text = cross_reference_mention.group(2)
  if not row["SemantK"] == "тожде" and not cross_reference_mention:
    note.text = row["SemantK"]
  if note_in_entry:
    note_editorial = etree.SubElement(note, "note", type="editor")
    note_editorial.text = note_in_entry.group(1)
